In [7]:
import random
import pandas as pd
import numpy as np
from Time_MoE.time_moe.datasets.time_moe_dataset import TimeMoEDataset

# Importing custom functions
import sys
import os
root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(root_path)

from baseline.functions import load_data,create_intervals,create_windows,smape,smape_chunked,sample_fraction
from sklearn.metrics import mean_absolute_error, mean_squared_error

def load_data_clean():
    ds = TimeMoEDataset('Time-300B\healthcare',normalization_method='zero')

    verbose = True
    total = len(ds)
    valid_indices = []
    # Iterar y filtrar
    for i in range(total):
        try:
            seq = ds[i]  # seq es numpy.ndarray según comprobaste
        except Exception as e:
            # Si hay error al obtener la secuencia, lo avisamos y saltamos
            if verbose:
                print(f"Advertencia: no se pudo obtener ds[{i}]: {e}")
            continue
        
        # Comprobación: si todos los valores son NaN, lo descartamos
        # seq es numpy.ndarray; cuidado si dims especiales, pero np.isnan funcionará elementwise.
        try:
            if not np.all(np.isnan(seq)):
                valid_indices.append(i)
        except Exception as e:
            # En caso de que seq no sea array puro, convertir primero:
            try:
                arr = np.array(seq)
                if not np.all(np.isnan(arr)):
                    valid_indices.append(i)
            except Exception as e2:
                if verbose:
                    print(f"Error al verificar NaN en secuencia índice {i}: {e2}")
                # Decidir si incluirla o no. Aquí optamos por descartarla:
                continue
    
    valid_count = len(valid_indices)
    if verbose:
        print(f"Secuencias totales en ds: {total}")
        print(f"Secuencias válidas (no todo NaN): {valid_count}")
        print(f"Secuencias descartadas: {total - valid_count}")
        sequences_validas = []

    for idx in valid_indices:
        try:
            sequences_validas.append(ds[idx])
        except Exception as e:
            if verbose:
                print(f"Error al extraer ds[{idx}] después de filtrar: {e}")
            # Podrías decidir saltar o detener. Aquí solo saltamos.
    return sequences_validas

def create_windows_from_sequences(sequences, window_size=15, horizon=1):
    """
    Dada una lista de secuencias (numpy arrays 1D), crea ventanas deslizantes:
    - X: array de shape (num_samples, window_size, 1)
    - y: array de shape (num_samples,)
    Cada muestra usa window_size pasos para predecir el siguiente valor (horizon=1).
    """
    X_list = []
    y_list = []
    for seq in sequences:
        # Asegurar numpy array
        arr = np.array(seq).astype(float)
        T = arr.shape[0]
        # Solo si la longitud es mayor que window_size + horizon - 1
        if T >= window_size + horizon:
            for start in range(0, T - window_size - horizon + 1):
                window = arr[start:start+window_size]
                target = arr[start+window_size:start+window_size+horizon]
                # Para horizon=1, target es un array de longitud 1; tomamos el escalar
                X_list.append(window.reshape(window_size, 1))
                y_list.append(target[0] if horizon == 1 else target)
    if len(X_list) == 0:
        return np.empty((0, window_size, 1)), np.empty((0,))
    X = np.stack(X_list, axis=0)
    y = np.array(y_list)

    # Supongamos X tiene forma (N, window_size, 1), y y forma (N,)
    mask_valid = ~np.isnan(X).any(axis=(1,2)) & ~np.isnan(y)
    # Mantener solo muestras sin NaN:
    X_clean = X[mask_valid]
    y_clean = y[mask_valid]
    print("De", X.shape[0], "muestras, quedan", X_clean.shape[0], "sin NaN")

    return X_clean, y_clean

In [2]:
ds = load_data_clean()
X, y = create_windows_from_sequences(ds, window_size=15, horizon=1)

Secuencias totales en ds: 1752
Secuencias válidas (no todo NaN): 1752
Secuencias descartadas: 0
De 433317 muestras, quedan 433317 sin NaN


### Neural - 100 %

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

In [12]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler , RobustScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from matplotlib.ticker import MaxNLocator
import torch
import torch.nn as nn
import torch.optim as optim
from torchdiffeq import odeint
import numpy as np
import pandas as pd
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import os

In [13]:
# Neural ODE Model
class ODEFunc(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(1, 50),
            nn.Tanh(),
            nn.Linear(50, 1)
        )

    def forward(self, t, x):
        return self.net(x)

class NeuralODEModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.odefunc = ODEFunc()
    
    def forward(self, x):
        # x: [batch_size, seq_len, features]
        device = x.device
        seq_len = x.shape[1]
        # Crear t en el dispositivo correcto
        t = torch.linspace(0, 1, seq_len, device=device)
        # Estado inicial para cada muestra: último valor de la secuencia
        # Si features=1, x[:, -1, :] es [batch_size, 1]
        y0 = x[:, -1, :]
        # Integrar en batch: devuelve [len(t), batch_size, features]
        out = odeint(self.odefunc, y0, t, method='rk4')
        # Tomar el valor final en t=1 para cada muestra
        y_final = out[-1]
        return y_final  # forma [batch_size, features]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Usa DataLoader para evitar cargar todo en la GPU al mismo tiempo
def train_model(model, X_train, y_train, X_test, y_test, batch_size=64, epochs=20):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    train_dataset = torch.utils.data.TensorDataset(
        torch.tensor(X_train, dtype=torch.float32),
        torch.tensor(y_train, dtype=torch.float32)
    )
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    model = model.to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    loss_vals = []
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0.0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            output = model(xb)
            loss = criterion(output, yb)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() * xb.size(0)
        epoch_loss /= len(train_loader.dataset)
        loss_vals.append(epoch_loss)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}")

    best_epoch = int(np.argmin(loss_vals)) + 1
    best_loss = loss_vals[best_epoch - 1]
    print(f"Mejor época = {best_epoch}, Loss = {best_loss:.4f}")

    # Evaluación
    model.eval()
    with torch.no_grad():
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
        y_test_tensor = torch.tensor(y_test, dtype=torch.float32).to(device)
        y_pred = model(X_test_tensor).cpu().numpy()

    y_test_inv = y_test_tensor.cpu().numpy().reshape(-1, 1)
    mae = mean_absolute_error(y_test_inv, y_pred)
    mse = mean_squared_error(y_test_inv, y_pred)
    smape_value = smape(y_test_inv, y_pred)
    print(f"MAE: {mae:.4f}, MSE: {mse:.4f}, sMAPE: {smape_value:.4f}")

    return model, loss_vals

In [15]:
model = NeuralODEModel()
trained_model, loss_vals = train_model(model, X_train, y_train, X_test, y_test)

c:\Users\criju\.conda\envs\cuda\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\criju\.conda\envs\cuda\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([29])) that is different to the input size (torch.Size([29, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/20, Loss: 1.0131
Epoch 2/20, Loss: 1.0036
Epoch 3/20, Loss: 1.0035
Epoch 4/20, Loss: 1.0035
Epoch 5/20, Loss: 1.0034
Epoch 6/20, Loss: 1.0034
Epoch 7/20, Loss: 1.0034
Epoch 8/20, Loss: 1.0034
Epoch 9/20, Loss: 1.0034
Epoch 10/20, Loss: 1.0034
Epoch 11/20, Loss: 1.0034
Epoch 12/20, Loss: 1.0034
Epoch 13/20, Loss: 1.0034
Epoch 14/20, Loss: 1.0034
Epoch 15/20, Loss: 1.0034
Epoch 16/20, Loss: 1.0034
Epoch 17/20, Loss: 1.0033
Epoch 18/20, Loss: 1.0034
Epoch 19/20, Loss: 1.0034
Epoch 20/20, Loss: 1.0033
Mejor época = 17, Loss = 1.0033
MAE: 0.7171, MSE: 0.9914, sMAPE: 97.4496


In [16]:
# Supongamos que tu modelo ya ha sido entrenado
torch.save(model.state_dict(), "Models_neural/modelo_100.pth")

### Neural - 10 %

In [17]:
# function to take only a percentage of the data
X_10, y_10 = sample_fraction(X, y, 0.10, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_10, y_10, test_size=0.2, shuffle=True, random_state=42)

In [18]:
model = NeuralODEModel()
trained_model, loss_vals = train_model(model, X_train, y_train, X_val, y_val)

c:\Users\criju\.conda\envs\cuda\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\criju\.conda\envs\cuda\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([40])) that is different to the input size (torch.Size([40, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/20, Loss: 1.1121
Epoch 2/20, Loss: 0.9849
Epoch 3/20, Loss: 0.9803
Epoch 4/20, Loss: 0.9787
Epoch 5/20, Loss: 0.9782
Epoch 6/20, Loss: 0.9779
Epoch 7/20, Loss: 0.9773
Epoch 8/20, Loss: 0.9775
Epoch 9/20, Loss: 0.9772
Epoch 10/20, Loss: 0.9771
Epoch 11/20, Loss: 0.9770
Epoch 12/20, Loss: 0.9771
Epoch 13/20, Loss: 0.9771
Epoch 14/20, Loss: 0.9770
Epoch 15/20, Loss: 0.9767
Epoch 16/20, Loss: 0.9767
Epoch 17/20, Loss: 0.9768
Epoch 18/20, Loss: 0.9768
Epoch 19/20, Loss: 0.9767
Epoch 20/20, Loss: 0.9767
Mejor época = 15, Loss = 0.9767
MAE: 0.7131, MSE: 0.9639, sMAPE: 97.2977


In [19]:
# Supongamos que tu modelo ya ha sido entrenado
torch.save(model.state_dict(), "Models_neural/modelo_10.pth")

### Neural - 3 %

In [20]:
# function to take only a percentage of the data
X_3, y_3 = sample_fraction(X, y, 0.03, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_3, y_3, test_size=0.2, shuffle=True, random_state=42)

In [21]:
model = NeuralODEModel()
trained_model, loss_vals = train_model(model, X_train, y_train, X_val, y_val)
torch.save(model.state_dict(), "Models_neural/modelo_3.pth")

c:\Users\criju\.conda\envs\cuda\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\criju\.conda\envs\cuda\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([31])) that is different to the input size (torch.Size([31, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/20, Loss: 1.2142
Epoch 2/20, Loss: 1.0117
Epoch 3/20, Loss: 0.9921
Epoch 4/20, Loss: 0.9858
Epoch 5/20, Loss: 0.9826
Epoch 6/20, Loss: 0.9807
Epoch 7/20, Loss: 0.9795
Epoch 8/20, Loss: 0.9787
Epoch 9/20, Loss: 0.9783
Epoch 10/20, Loss: 0.9778
Epoch 11/20, Loss: 0.9772
Epoch 12/20, Loss: 0.9772
Epoch 13/20, Loss: 0.9769
Epoch 14/20, Loss: 0.9764
Epoch 15/20, Loss: 0.9765
Epoch 16/20, Loss: 0.9763
Epoch 17/20, Loss: 0.9764
Epoch 18/20, Loss: 0.9758
Epoch 19/20, Loss: 0.9756
Epoch 20/20, Loss: 0.9765
Mejor época = 19, Loss = 0.9756
MAE: 0.7114, MSE: 0.9644, sMAPE: 98.2612


# Neural Destilado